In [149]:
import requests
import json
import pandas as pd
import csv

In [150]:
url = "https://api.annict.com/graphql"

token = input() # Annictのアクセストークン
headers = {"Authorization": f"bearer {token}"}

query = """query {
            viewer {
              works(state: WATCHED, orderBy:{field: SEASON, direction:DESC}) {
                nodes {
                  title,
                  seriesList {
                    nodes {
                      name,
                      works(orderBy:{field:SEASON, direction:ASC}) {
                        edges {
                          summary,
                          node {
                            viewerStatusState,
                            title
                          }
                        }
                      }
                    }
                  }
                }
              }
            }
          }"""

In [151]:
r = requests.post(url, headers=headers, data={"query": query})
print(r.status_code)

df = pd.DataFrame(json.loads(r.text)["data"]["viewer"]["works"]["nodes"])

200


In [152]:
notViewed = []
set_notViewd = set()
ignore_list = ["プリキュア", "ルパン", "ポケットモンスター", "コナン", "ドラゴンボール"]

for index, row in df.iterrows():
  if any(x in row["title"] for x in ignore_list): continue

  df2 = pd.DataFrame(row["seriesList"]["nodes"])
  for index2, row2 in df2.iterrows():
    seriesName = row2["name"]

    df3 = pd.DataFrame(row2["works"]["edges"])
    for index3, row3, in df3.iterrows():
      status = row3["node"]["viewerStatusState"]
      title = row3["node"]["title"]
      summary = row3["summary"]

      if status == "NO_STATE":
        if (all(x not in title for x in ignore_list)) and (title not in set_notViewd):
            notViewed.append({f"{seriesName}({summary})": title})
            set_notViewd.add(title)

In [153]:
with open("./notViewed_anime.csv", "w") as f:
  writer = csv.writer(f)
  for anime in notViewed:
    for summary, title in anime.items():
      writer.writerow([summary, title])
      print(f"{summary}: {title}")

ゆるキャン△(DVD/Blu-ray特典映像): ゆるキャン△ 映像特典
ゆるキャン△(へやキャン△ DVD/Blu-ray映像特典): へやキャン△ SPECIAL EPISODE「サウナとごはんと三輪バイク」
鬼滅の刃(総集編1): 鬼滅の刃 兄妹の絆
鬼滅の刃(総集編3): 鬼滅の刃 柱合会議・蝶屋敷編
鬼滅の刃(総集編2): 鬼滅の刃 那田蜘蛛山編
鬼滅の刃(特別編集版): 鬼滅の刃 竈門炭治郎 立志編 特別編集版
PUI PUI モルカー(劇場版): とびだせ！ならせ！PUI PUI モルカー
ダンジョンに出会いを求めるのは間違っているだろうか(OVA): ダンジョンに出会いを求めるのは間違っているだろうか III OVA「オラリオに温泉を求めるのは間違っているだろうか 〜おふろの神様フォーエバー〜」
やはり俺の青春ラブコメはまちがっている。(OVA): やはり俺の青春ラブコメはまちがっている。完 OVA
かぐや様は告らせたい〜天才たちの恋愛頭脳戦〜(OVA): かぐや様は告らせたい〜天才たちの恋愛頭脳戦〜 OVA 「かぐや様ダークネスvolume 1／かぐや様ダークネスvolume 2／かぐや様は食べさせたい」
この素晴らしい世界に祝福を！(OVA): この素晴らしい世界に祝福を！2 OVA「この素晴らしい芸術に祝福を！」
HELLO WORLD(スピンオフ): ANOTHER WORLD
あそびあそばせ(OVA): あそびあそばせ OVA「コスプレ大会／大人の私へ」
あそびあそばせ(OVA): あそびあそばせ 新作アニメ
ヲタクに恋は難しい(OVA): ヲタクに恋は難しい OAD「Youth」
ヲタクに恋は難しい(OVA): ヲタクに恋は難しい OAD 第2話「トモダチの距離」
ヲタクに恋は難しい(OVA): ヲタクに恋は難しい 11巻特装版付属​OAD「社員旅行と願い事」
ソードアート・オンライン(4期): ソードアート・オンライン アリシゼーション War of Underworld
ソードアート・オンライン(4期2部): ソードアート・オンライン アリシゼーション War of Underworld ‐THE LAST SEASON‐
ソードアート・オンライン(): 劇場版 ソードアート・オンライン プログレッシブ 星なき夜のアリア
暗殺教室(O